In [1]:
import nltk
from nltk import WordNetLemmatizer
import pandas as pd
import os
import re

In [2]:
cwd=os.getcwd()
path_to_raw_texts=os.path.join(cwd,"texts/raw_texts")
preprocessed_texts_file=os.path.join(cwd,"texts/preprocessed_texts_for_doc2vec.pkl")
stopwords=['tbsp','tablespoon','tsp','teaspoon','g','gram','kg','f','l','ml','t','s',
           'sec','m','minute','h','hour','%','eel','pre','lb','oz',
           '/','‘','‘‘',':','°','inch','pound','pint','cup','ingridients','method',]
#nltk can't lemmatize word 'ingridients', so it is in plural in the list of stopwords

labels=[] #food category 
text_names=[] #(1.txt, 2.txt, etc)
preprocessed_texts=[]

corpus = nltk.corpus.reader.plaintext.PlaintextCorpusReader(path_to_raw_texts,'.*\.txt')
abspaths=corpus.abspaths()
lemmatizer = WordNetLemmatizer()

for path_of_raw_recipe in abspaths:
    
    splitted_path=re.split('\/',path_of_raw_recipe)
    length_of_path=len(splitted_path)
    food_category=splitted_path[length_of_path-2]
    labels.append(food_category)
    text_name=splitted_path[length_of_path-1]
    text_names.append(text_name)
    
    recipe_text=corpus.words(path_of_raw_recipe)
    
    text_without_numbers=[]
    for word in recipe_text:
        text_without_numbers.append(re.sub(r'\d+', '', word)) #sushi2 -> sushi
        
    lowercase_text=' '.join(text_without_numbers).lower()
    tokens = nltk.word_tokenize(lowercase_text)
    tags = nltk.pos_tag(tokens)
    nouns = [word for word,pos in tags if (pos == 'NN'  or  pos == 'NNS')]
    
    l_nouns=[]
    for n in nouns:
        l_nouns.append(lemmatizer.lemmatize(n))
        
    filtered_nouns = [l_noun for l_noun in l_nouns if l_noun not in stopwords]
    preprocessed_texts.append(filtered_nouns)
    

    
dict_preprocessed_texts = {
    'text_names': text_names,
    'labels' : labels,
    'preprocessed_texts' : preprocessed_texts   
}
    

df_preprocessed_texts=pd.DataFrame(dict_preprocessed_texts)

print("Saving preprocessed texts pandas dataframe to: ", preprocessed_texts_file)
df_preprocessed_texts.to_pickle(preprocessed_texts_file)

print("Preprocessed texts pandas dataframe: \n",df_preprocessed_texts)

Saving preprocessed texts pandas dataframe to:  /notebooks/school/text_feature_extraction/texts/preprocessed_texts_for_doc2vec.pkl
Preprocessed texts pandas dataframe: 
       labels                                 preprocessed_texts text_names
0    sashimi  [salmon, avocado, onion, clove, wasabi, seed, ...      1.txt
1    sashimi  [person, fresh, grade, tuna, steak, fillet, wa...     10.txt
2    sashimi  [sashimi, grade, salmon, filet, pure, sesame, ...      2.txt
3    sashimi  [sushi, filet, block, sushi, filet, block, cor...      3.txt
4    sashimi  [handful, coriander, leaf, sesame, seed, oil, ...      4.txt
5    sashimi  [sushi, filet, straight, grain, water, salt, s...      5.txt
6    sashimi  [sushi, sirloin, coriander, key, tuna, sashimi...      6.txt
7    sashimi  [grade, tuna, avocado, cut, slice, oil, lime, ...      7.txt
8    sashimi  [grade, tuna, piece, piece, ginger, mirin, sak...      8.txt
9    sashimi  [sushi, rice, milliliter, rice, vinegar, vineg...      9.txt
10   